In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
sns.set_context('notebook')

RANDOM_SEED = 20090425

# Classical Statics

In [2]:
data = pd.read_csv('data.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data.csv'

In [ ]:
from tokenize import group


Male = pd.DataFrame(dict(bodyTemp=(96.7,96.9,97,97.1,97.1,97.1,97.2,97.3,97.4,97.4,97.4,97.4,97.5,97.5,97.6,97.6,97.6,97.7,97.8,97.8,97.8,97.8,97.9,97.9,98,98,
98,98,98,98,98.1,98.1,98.2,98.2,98.2,98.2,98.3,98.3,98.4,98.4,98.4,98.4,98.5,98.5,98.6,98.6,98.6,98.6,98.6,98.6,98.7,98.7,98.8,98.8,98.8,98.9,99,99,99,99.1,99.2
,99.3,99.4,99.5),group='Male'))

Female = pd.DataFrame(dict(bodyTemp=(96.7,96.8,97.2,97.2,97.4,97.6,97.7,97.7,97.8,97.8,97.8,97.9,97.9,97.9,98,98,98,98,98,98.1,98.2,98.2,98.2,98.2,98.2,98.2,98.3,98.3,98.3,98.4,
98.4,98.4,98.4,98.4,98.5,98.6,98.6,98.6,98.6,98.7,98.7,98.7,98.7,98.7,98.7,98.8,98.8,98.8,98.8,98.8,98.8,98.8,98.9,99,99,99.1,99.1,99.2,99.2,99.3,99.4,99.9,100),group='Female'))

Male_Heart_Rate=pd.DataFrame(dict(bodyTemp=()));
Female_Heart_Rate=pd.DataFrame(dict(bodyTemp=()));

In [ ]:
from tokenize import group
trial_data = pd.concat([Male, Female], ignore_index=True)
trial_data.hist('bodyTemp', by='group')

### T-test for comparind Male/Female body-Temprature outcomes

In [ ]:
import scipy.stats as stats
combined_temp = np.concatenate((Male.bodyTemp, Female.bodyTemp))
print(len(Male.bodyTemp))
print(len(Female.bodyTemp))
print( Male.bodyTemp.mean() )
print( Female.bodyTemp.mean() )
print( combined_temp.mean() )
print(Male.bodyTemp.std())
stats.ttest_ind(a= Male.bodyTemp,
                b= Female.bodyTemp,
                equal_var=True) 

# Bayesian Statitcs

In [ ]:
pip install pymc3

In [ ]:
from pymc3 import Model, Normal
with Model() as Temprature_model:
    μ_0 = Normal('μ_0', 100, sd=10)
    μ_1 = Normal('μ_1', 100, sd=10)

In [ ]:
from pymc3 import Model, Uniform
with Model() as Temprature_model:
    σ_0 = Uniform('σ_0', lower=1, upper=10)
    σ_1 = Uniform('σ_1', lower=1, upper=10)

In [ ]:
from pymc3 import Model, Uniform
with Model() as Temprature_model:
    ν = Normal('ν', 100,sd=5) 

In [ ]:
from pymc3 import StudentT
with Temprature_model:
    Male = StudentT('Temprature_Male', nu=ν, mu=μ_1, lam=σ_1**2,observed= Male.bodyTemp)
    Female = StudentT('Temprature_Female', nu=ν, mu=μ_0, lam=σ_0**2,observed= Female.bodyTemp)

In [ ]:
from pymc3 import Deterministic
    
with Temprature_model:
    diff_of_means = Deterministic('difference of means', μ_1 - μ_0)
    effect_size = Deterministic('effect size', diff_of_means / np.sqrt((σ_1**2 + σ_0**2) / 2))

In [ ]:
from pymc3 import fit

with Temprature_model: 
    Temp_fit = fit(random_seed=RANDOM_SEED).sample(1000)

In [ ]:
from pymc3 import plot_posterior

plot_posterior(Temp_fit[100:], 
                var_names=['μ_0', 'μ_1', 'σ_0', 'σ_1', 'ν'],
                color='#87ceeb');